<a href="https://colab.research.google.com/github/sdeutchman/watson/blob/Akira/Watson_Elementary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import library 

In [ ]:
import numpy as np
import pandas as pd

# Download data

In [ ]:
train = pd.read_csv('/kaggle/input/contradictory-my-dear-watson/train.csv')

# Exploratory data analysis

In [ ]:
train.info()

# Preparing data for input

In [ ]:
#strip
#removes all the whitespaces at the beginning and the end of a string
texts = [
    '   Bonjour, comment ca va ?     ',
    '    Heyyyyy, how are you doing ?   ',
    '        Hallo, wie gehts ?     '
]
[text.strip() for text in texts]

In [ ]:
#replace
text = "I love koalas, koalas are the cutest animals on Earth."
text.replace("koala", "panda")

In [ ]:
#splitting
text = "linkin park / metallica /red hot chili peppers"
text.split("/")

In [ ]:
#lowercase
text = "i LOVE football sO mUch. FOOTBALL is my passion. Who else loves fOOtBaLL ?"
text.lower() 

In [ ]:
#dealing with numbers, punctuation, and symbols

#numbers
text = "i do not recommend this restaurant, we waited for so long, like 30 minutes, this is ridiculous"
cleaned_text = ''.join(char for char in text if not char.isdigit())

#punctuation and symbols
import string # "string" module is already installed with Python
string.punctuation
for punctuation in string.punctuation:
    text = text.replace(punctuation, '') 



In [ ]:
#tokeninzing
text = 'It is during our darkest moments that we must focus to see the light'
from nltk.tokenize import word_tokenize
word_tokens = word_tokenize(text)

In [ ]:
#removing stopwords
from nltk.corpus import stopwords 

tokens = ["i", "am", "going", "to", "go", "to", "the", 
        "club", "and", "party", "all", "night", "long"]

stop_words = set(stopwords.words('english')) 
tokens_cleaned = [w for w in tokens if not w in stop_words] 

In [ ]:
#Lemmatizing
#Lemmatizing is a technique used to find the root of words, in order to group them by their meaning rather than by their exact form

# Lemmatizing the verbs
verb_lemmatized = [                  
    WordNetLemmatizer().lemmatize(word, pos = "v") # v --> verbs
    for word in tokenized_sentence_no_stopwords
]

# 2 - Lemmatizing the nouns
noun_lemmatized = [                 
    WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
    for word in verb_lemmatized
]

# High dimensional Embedding

In [ ]:
#Each word is represented by a vector of chosen length